In [1]:
import numpy as np
import scipy.signal
import cv2
import csv
import math
import time

In [2]:
# each track in the dataset is in a folder titled foldername
# the track notes, patterns, and metadata are present in the folder 

foldername = 'africa_bass'

In [5]:
# read metadata from file 
# tempo, bpm, ticks per beat, pitch tolerance, amplitude tolerance, duration tolerance, number of patterns

met = list(csv.reader(open('data/csv/'+foldername+'/metadata.csv')))
tempo = int(met[0][0])
bpm = int(met[1][0])
tpb = int(met[2][0])
t_ptc =int(met[3][0])
t_amp = int(met[4][0])
t_dur = float(met[5][0])
num_patterns = int(met[6][0])

In [6]:
# read the patterns from file. construct labels for patterns

pat = []
lbl = []
for i in range (0, num_patterns):
 lbl.append(i+1)
 patfile = 'data/csv/'+foldername+'/p' + str(i+1) + '.csv'
 pt = []
 with open(patfile) as csvfile:
  rdr = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
  for row in rdr:
   pt.append(row)
 pat.append(pt)

In [7]:
# read csv file of the midi data in matrix format

test_data = []
with open('data/csv/'+foldername+'/data.csv') as csvfile:
 rdr = csv.reader(csvfile, quoting=csv.QUOTE_NONNUMERIC)
 for row in rdr:
  test_data.append(row)

In [8]:
def ssim(img1, img2):
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)    
    kernel = cv2.getGaussianKernel(3, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)  # valid
    mu2 = cv2.filter2D(img2, -1, window)
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window) - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window) - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window) - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    return abs(ssim_map.mean())
   

In [9]:
def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 255]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1, img2))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [12]:
def get_ssim(arr1, arr2):
 note_1 = arr1[:,0]
 note_2 = arr2[:,0]
 pp_1 = arr1[:,1:3]
 pp_2 = arr2[:,1:3]
 
 #-----------------------------------------------------------------------------
 #this approach can detect transposed patterns
 corr = np.corrcoef(note_1, note_2)
 det = np.linalg.det(np.asarray(corr))
 if det==0:
  det = 1
 else:
  det = 0 
 ssim_val = calculate_ssim(pp_1, pp_2) 
 return det*ssim_val
 
#  #-----------------------------------------------------------------------------
#  #to desregard transposed patterns
#  det = ssim(note_1, note_2) 
#  if det!=1:
#   det = 0
#  ssim_val = calculate_ssim(pp_1, pp_2) 
#  return det*ssim_val

In [11]:
test_data_len = len(test_data)
for i in range(0, test_data_len):
 ppt = []
 for j in range(0, len(pat)):
#   if (i+int(len(pat[j])) < test_data_len):
  if (test_data_len - (i+len(pat[j]))) > 0 :
   nt = test_data[i:i+int(len(pat[j]))]
   ppt.append(get_ssim(np.asarray(nt),np.asarray(pat[j])))
  else:
   ppt.append(0)
 print(i+1, np.asarray(ppt))
#  print(i, ppt)
 

0.5183909698184028 0.04466853837589586
0.589628522242134 0.7891430516944956
0.9898151888398735 0.35632818827238966
1 [0. 0. 0.]
0.0 0.9998246687621261
0.5922722197727677 0.040868006917626314
0.9645354494894519 0.5335477481208116
2 [0.99982467 0.         0.        ]
0.695296842158213 0.06858836099380486
0.7770624246860961 0.6706568248502084
0.9919053364891146 0.4029432111089158
3 [0. 0. 0.]
0.9465614676935238 0.5640383241932834
0.8628151890354167 0.18710829270858653
0.9912891921333139 0.37881220657406894
4 [0. 0. 0.]
0.9421690907645485 0.32500366328519725
0.9851520017662848 0.4951500937366453
0.9997411467520952 0.467141689313889
5 [0. 0. 0.]
0.9813292296663276 0.3898256140259967
0.9993670647383049 0.3832726779456996
0.9984783245781378 0.4484588523013804
6 [0. 0. 0.]
0.9608145756390352 0.5514810478300999
0.9939251068017076 0.46893972735461587
0.9987991083534734 0.4548973616150055
7 [0. 0. 0.]
0.9608145756390352 0.5358293960453477
0.9858568495212675 0.474918345895779
0.9974536191594139 0.

0.6035787331590805 0.7927131228789883
0.7535899657641553 0.14434036595749677
103 [0. 0. 0.]
0.01234998144440957 0.9998000446992069
0.6181266267792191 0.04044566711296669
0.9368442299284294 0.5838829645149524
104 [0. 0. 0.]
0.7266397108514344 0.06812142054292242
0.8688072988381828 0.5898753200671215
0.9802283230867318 0.35578107647303137
105 [0. 0. 0.]
0.9985303855803195 0.47418940386062514
0.9958446993028871 0.3782073127336993
0.9760953550129329 0.3366099428327483
106 [0. 0. 0.]
0.9482215474621916 0.5354018357364967
0.88814850728371 0.18221919194802064
0.8690600440961493 0.6351265194877262
107 [0. 0. 0.]
0.5136109409431813 0.04685191416217252
0.5970830651798453 0.7927131228789883
0.9359706782471339 0.292295324789251
108 [0. 0. 0.]
0.005615172594297115 0.9998000446992069
0.6068288151011652 0.04044566711296669
0.9811469505832983 0.514654823157818
109 [0. 0. 0.]
0.7177469422563718 0.06812142054292242
0.7796403375599914 0.6705986579014688
0.9830622559167016 0.3557810764730313
110 [0. 0. 0.

0.9959757549970898 0.46085034608157427
210 [0. 0. 0.]
0.9919703042416627 0.3970553594670523
0.8845964307125499 0.5950198779996441
0.9165932681805586 0.3874653477356911
211 [0. 0. 0.]
0.9997082015574065 0.48153350706805126
0.6289689375786203 0.11139863352520579
0.980975875863263 0.5493313865701417
212 [0. 0. 0.]
0.7300896974551114 0.2424101429403661
0.48680001555095354 0.7955542764241104
0.9956539810968474 0.5217578561437788
213 [0. 0. 0.]
0.8261824795792475 0.7174575532981758
0.7055691267691188 0.18896023996253056
0.9766745145296684 0.5883308487848778
214 [0. 0. 0.]
0.9189484272869262 0.33388253502716514
0.6690812519499871 0.6487003020211906
0.9990919033017298 0.4981454841292437
215 [0. 0. 0.]
0.6312525303343852 0.7558237055077481
0.7829796314613948 0.28250973020145936
0.8371580337382142 0.3485111476985655
216 [0. 0. 0.]
0.852392345992691 0.3334032230504994
0.9998718610934891 0.4249838960763781
0.9733487722488056 0.5941736407970938
217 [0. 0. 0.]
0.8038513070087754 0.3115918489998943
0

0.6565121565987437 0.11707741296557263
0.31460721015417037 0.8733501280793163
0.992719599408804 0.38561916932277845
311 [0. 0. 0.]
0.030603054166885472 0.9999975009860261
0.641420878887665 0.039623216913352084
0.9661116398685998 0.5333319006798173
312 [0. 0. 0.]
0.7501957782110087 0.06706529766408055
0.8031119209462336 0.6699142582751237
0.9964424451817553 0.40249475551290853
313 [0. 0. 0.]
0.9587673158299604 0.5632233577905043
0.8956160018171291 0.18583098386551297
0.989261180522703 0.37835888030941994
314 [0. 0. 0.]
0.9701964144625784 0.3236102354563898
0.9942560585770854 0.49385627791143527
0.9998595376072096 0.4670217884842428
315 [0. 0. 0.]
0.9653162343636406 0.388414178580313
0.9996316220042559 0.38293101166357574
0.9989734815131815 0.4480198971870795
316 [0. 0. 0.]
0.972798479140875 0.5511083209768739
0.9797175955624391 0.4677005694420351
0.9990377512555413 0.45464101687574227
317 [0. 0. 0.]
0.9215343380407857 0.5344775874134414
0.9942560585770854 0.47417601314323515
0.992912363

0.9546462625901746 0.5354018357364967
0.8974220184319952 0.18221919194802064
0.6427955722609912 0.8013043178567534
412 [0. 0. 0.]
0.5303502818352979 0.04685191416217252
0.6035787331590805 0.7927131228789883
0.7535899657641553 0.14434036595749677
413 [0. 0. 0.]
0.01234998144440957 0.9998000446992069
0.6181266267792191 0.04044566711296669
0.9368442299284294 0.5838829645149524
414 [0. 0. 0.]
0.7266397108514344 0.06812142054292242
0.8688072988381828 0.5898753200671215
0.9802283230867318 0.35578107647303137
415 [0. 0. 0.]
0.9985303855803195 0.47418940386062514
0.9958446993028871 0.3782073127336993
0.9760953550129329 0.3366099428327483
416 [0. 0. 0.]
0.9482215474621916 0.5354018357364967
0.88814850728371 0.18221919194802064
0.8690600440961493 0.6351265194877262
417 [0. 0. 0.]
0.5136109409431813 0.04685191416217252
0.5970830651798453 0.7927131228789883
0.9359706782471339 0.292295324789251
418 [0. 0. 0.]
0.005615172594297115 0.9998000446992069
0.6068288151011652 0.04044566711296669
0.981146950

...

0.9985303855803195 0.47418940386062514
0.9958446993028871 0.3782073127336993
0.8743303893088359 0.23545203591156566
679 [0. 0. 0.]
0.9482215474621916 0.5354018357364967
0.88814850728371 0.18221919194802064
0.9267611372938693 0.6043420494595294
680 [0. 0. 0.]
0.5136109409431813 0.04685191416217252
0.5970830651798453 0.7927131228789883
0.9920610560940583 0.4711723170652012
681 [0. 0. 0.]
0.005615172594297115 0.9998000446992069
0.6068288151011652 0.04044566711296669
0.8838063886264657 0.27287058568679823
682 [0. 0. 0.]
0.7177469422563718 0.06812142054292242
0.7796403375599914 0.6705986579014688
0.9114605768607416 0.6143032746677993
683 [0. 0. 0.]
0.9451857173784028 0.5640195686050149
0.877912271444421 0.18687756979605566
0.9574977776596341 0.3046801772687001
684 [0. 0. 0.]
0.9490584319389146 0.3249626844425206
0.9890207715397145 0.49493952455003437
0.9998110907598793 0.4635336836184983
685 [0. 0. 0.]
0.9796461784531979 0.38959148372582836
0.9914690675789255 0.47326289739858757
0.888026599

0.9491714859873885 0.39982551308374853
747 [0. 0. 0.]
0.907888405244896 0.3334032230504994
0.9977353896451948 0.4249838960763781
0.9989994202595905 0.5130061086903634
748 [0. 0. 0.]
0.7489639430563088 0.3115918489998943
0.9300984327800667 0.6052565899028594
0.9806174334397767 0.5440989597148783
749 [0. 0. 0.]
0.597964772977183 0.7880705691295816
0.8363979354532963 0.27071959819162256
0.9982887424118677 0.5137476904308244
750 [0. 0. 0.]
0.8982983283971816 0.33481420362952563
0.7501516953258651 0.649739307677059
0.9961983717130566 0.5227115821234274
751 [0. 0. 0.]
0.6200361650908066 0.75598452294854
0.8790222669209391 0.28330231298985414
0.9971256651625134 0.46349663025256466
752 [0. 0. 0.]
0.8793400277688711 0.3335072985289422
0.998114832228664 0.5057513329346333
0.9865463047651375 0.4718240052662233
753 [0. 0. 0.]
0.9027910352219177 0.40147734031605303
0.9988811312627978 0.41397972010625655
0.9769734794099412 0.5706160729478063
754 [0. 0. 0.]
0.9982202247551395 0.5777204893240694
0.992

0.8277164912434981 0.26890895928937975
0.9527931832372651 0.30846213514663073
0.9837196261569486 0.43052759137946783
874 [0. 0. 0.]
0.9909245125315109 0.46261642973586026
0.9049677577732832 0.5803618025560674
0.7976151546322945 0.29818203980792607
875 [0. 0. 0.]
0.8689533034107442 0.672974640347912
0.6581789773322549 0.0720432492869803
0.7047486007857093 0.748589820192735
876 [0. 0. 0.]
0.6621474243988298 0.11685403199214782
0.2991295424973406 0.8734095522134718
0.926042010203631 0.3358254261508464
877 [0. 0. 0.]
0.014105568033226492 0.9998040330217649
0.5879767851066116 0.040937944100512554
0.940484237665786 0.6104941043471874
878 [0. 0. 0.]
0.7096897248039652 0.0685780736223937
0.7687824229081774 0.6707196481180869
0.9979716927202533 0.4777742196518575
879 [0. 0. 0.]
0.9469873577431367 0.56411532072723
0.9040131080608729 0.2678322673662025
0.8017088480297475 0.2521665618866481
880 [0. 0. 0.]
0.9967208061738516 0.4148367212079134
0.9773332717652352 0.3037838666021989
0.710400275620471

0.9222241564437726 0.26903135122705635
0.9871419912451509 0.5851616012997662
0.931043997111755 0.3192690765612022
943 [0. 0. 0.]
0.909617253223841 0.7733914768678268
0.949644656797126 0.2611676853373961
0.8471063377202767 0.6984969139753257
944 [0. 0. 0.]
0.9584061504252739 0.3092567696573198
0.8557019506353044 0.6953031713629185
0.9714563177937453 0.4295525762343394
945 [0. 0. 0.]
0.9886149264796075 0.5909290289226361
0.9204452199393582 0.29231216248061564
0.9984245809035426 0.474721656777008
946 [0. 0. 0.]
0.9237670606729993 0.44151693979320805
0.9449513602047694 0.6049467806325767
0.9936757904070131 0.5608009878365265
947 [0. 0. 0.]
0.9697683652562552 0.509615030068475
0.9553070748141219 0.4134375185338071
0.9532535263196399 0.40092473725053834
948 [0. 0. 0.]
0.9278250910925787 0.5793218710082785
0.9929520141472464 0.4943668567781567
0.9620261469053664 0.6198817047857108
949 [0. 0. 0.]
0.9715170656251728 0.5793630221990025
0.8601042095655853 0.49441931362207026
0.9936465016069443 0.

0.6698926876781118 0.7700543372827755
1010 [0. 0. 0.]
0.8057620388852159 0.35721987306313036
0.6286707004820016 0.7018855991184426
0.7947730720729356 0.3253784372370244
1011 [0. 0. 0.]
0.5194744363382221 0.8055264520401184
0.8120105660674738 0.27386035305669837
0.9960165359842647 0.5109138840075688
1012 [0. 0. 0.]
0.8510190155497084 0.3382633515700238
0.9989820228255047 0.3889730174761708
0.9837279347215833 0.5591060657539068
1013 [0. 0. 0.]
0.9923003531507891 0.462516158129798
0.9275844277547205 0.5802659620161742
0.7648587021964127 0.2603445428393726
1014 [0. 0. 0.]
0.8806724694955199 0.6728709877427815
0.6730284372726985 0.07209960925535967
0.5757143963074869 0.8328419838756762
1015 [0. 0. 0.]
0.6178383367844809 0.11693295554152834
0.33493244048160054 0.8733194520773032
0.8019373581447853 0.2695018905478276
1016 [0. 0. 0.]
0.030603054166885472 0.9998943146384369
0.6101627786705526 0.04060673845992718
0.9871502622674113 0.5239523344479345
1017 [0. 0. 0.]
0.7083674987721911 0.06828757